# Week 4: Naive Bayes Classification (Part 2)

### Overview
This notebook builds on the activities in the previous notebooks on sentiment analysis.  In this lab, we will be putting everything together.  You will be focussing on the movie_review corpus us with a view to investigating:

- Evaluation metrics for classifier performance
- Which is the best classifier from a set of possibilities on a given test set
- What is the impact of varying training data size? To what extent does increasing the quantity of training data improve classifier performance?



### Preliminaries 

In [14]:
import nltk
nltk.download('movie_reviews')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import movie_reviews

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


>To access functionality defined in previous notebooks, copy the classes and functions defined in Week3Labs and Week4Labs into a `utils.py` file and then import it into the notebook.  There is a `utils.py` file included with these resources which you can update.  You should make sure that your classifier and ConfusionMatrix classes are defined in this file

In [2]:
#uncomment on google colab
from google.colab import drive
drive.mount('/content/drive')

#you don't need this on jupyter notebook unless you want to import from a directory which is not the current working directory
import sys
sys.path.append('/content/drive/My Drive/NLE Notebooks 2021/Week4LabsSolutions/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#import code to setup training and testing data, wordlist classifiers and NB classifiers

from utils import *

### Evaluating a Naïve Bayes classifier on test data
We are now ready to run our Naïve Bayes classifier on a set of test data. When we do this we want to return the accuracy of the classifier on that data, where accuracy is calculated as follows:

$$\frac{\mbox{number of test documents that the classifier classifiers correctly}}
{\mbox{total number of test documents}}$$

In order to compute this accuracy score, we need to give the classifier **labelled** test data.
- This will be in the same format as the training data.

>In the cell below, we set up 5 test documents in the class `weather` and 5 documents in the class `football`.

>Run this cell.

In [4]:
weather_sents_train = [
    "today it is raining",
    "looking cloudy today",
    "it is nice weather",
]

football_sents_train = [
    "city looking good",
    "advantage united",
]

weather_data_train = [(FreqDist(sent.split()), "weather") for sent in weather_sents_train] 
football_data_train = [(FreqDist(sent.split()), "football") for sent in football_sents_train]
train_data = weather_data_train + football_data_train

weather_sents_test = [
    "the weather today is nice",
    "it is raining cats and dogs",
    "the weather here is wet",
    "it was hot today",
    "rain due tomorrow",
]

football_sents_test = [
    "what a great goal that was",
    "poor defending by the city center back",
    "wow he missed a sitter",
    "united are a shambles",
    "shots raining down on the keeper",
]

weather_data_test = [(FreqDist(sent.split()), "weather") for sent in weather_sents_test] 
football_data_test = [(FreqDist(sent.split()), "football") for sent in football_sents_test]
test_data = weather_data_test + football_data_test



In [5]:
train_data

[(FreqDist({'is': 1, 'it': 1, 'raining': 1, 'today': 1}), 'weather'),
 (FreqDist({'cloudy': 1, 'looking': 1, 'today': 1}), 'weather'),
 (FreqDist({'is': 1, 'it': 1, 'nice': 1, 'weather': 1}), 'weather'),
 (FreqDist({'city': 1, 'good': 1, 'looking': 1}), 'football'),
 (FreqDist({'advantage': 1, 'united': 1}), 'football')]

In [6]:
test_data

[(FreqDist({'is': 1, 'nice': 1, 'the': 1, 'today': 1, 'weather': 1}),
  'weather'),
 (FreqDist({'and': 1, 'cats': 1, 'dogs': 1, 'is': 1, 'it': 1, 'raining': 1}),
  'weather'),
 (FreqDist({'here': 1, 'is': 1, 'the': 1, 'weather': 1, 'wet': 1}), 'weather'),
 (FreqDist({'hot': 1, 'it': 1, 'today': 1, 'was': 1}), 'weather'),
 (FreqDist({'due': 1, 'rain': 1, 'tomorrow': 1}), 'weather'),
 (FreqDist({'a': 1, 'goal': 1, 'great': 1, 'that': 1, 'was': 1, 'what': 1}),
  'football'),
 (FreqDist({'back': 1,
            'by': 1,
            'center': 1,
            'city': 1,
            'defending': 1,
            'poor': 1,
            'the': 1}),
  'football'),
 (FreqDist({'a': 1, 'he': 1, 'missed': 1, 'sitter': 1, 'wow': 1}), 'football'),
 (FreqDist({'a': 1, 'are': 1, 'shambles': 1, 'united': 1}), 'football'),
 (FreqDist({'down': 1,
            'keeper': 1,
            'on': 1,
            'raining': 1,
            'shots': 1,
            'the': 1}),
  'football')]

### Exercise 1
Train the NB classifier that you developed earlier and then test it.
Compute accuracy, precision, recall and F1 score

In [7]:
docs,goldstandard=zip(*test_data)

In [8]:
docs

(FreqDist({'is': 1, 'nice': 1, 'the': 1, 'today': 1, 'weather': 1}),
 FreqDist({'and': 1, 'cats': 1, 'dogs': 1, 'is': 1, 'it': 1, 'raining': 1}),
 FreqDist({'here': 1, 'is': 1, 'the': 1, 'weather': 1, 'wet': 1}),
 FreqDist({'hot': 1, 'it': 1, 'today': 1, 'was': 1}),
 FreqDist({'due': 1, 'rain': 1, 'tomorrow': 1}),
 FreqDist({'a': 1, 'goal': 1, 'great': 1, 'that': 1, 'was': 1, 'what': 1}),
 FreqDist({'back': 1,
           'by': 1,
           'center': 1,
           'city': 1,
           'defending': 1,
           'poor': 1,
           'the': 1}),
 FreqDist({'a': 1, 'he': 1, 'missed': 1, 'sitter': 1, 'wow': 1}),
 FreqDist({'a': 1, 'are': 1, 'shambles': 1, 'united': 1}),
 FreqDist({'down': 1,
           'keeper': 1,
           'on': 1,
           'raining': 1,
           'shots': 1,
           'the': 1}))

In [9]:
goldstandard


('weather',
 'weather',
 'weather',
 'weather',
 'weather',
 'football',
 'football',
 'football',
 'football',
 'football')

In [10]:
myNB=NBClassifier()
myNB.train(train_data)

In [11]:
cm=ConfusionMatrix(myNB.classify_many(docs),goldstandard,classes=("weather","football"))
cm.precision()

0.625

### Exercise 2
Now, we want to run your NB classifier on a real problem - the classification of movie reviews as positive or negative.
* generate a training and test split of the data for movie_reviews (see Lab 3_1 / 3_2)
* train a nb_classifier on the training data
* test it on the test data

Compare the performance of your Naive Bayes classifier with the WordList classifiers that you developed last week.

In [12]:

def get_train_test_data():
    
    #get ids of positive and negative movie reviews
    pos_review_ids=movie_reviews.fileids('pos')
    neg_review_ids=movie_reviews.fileids('neg')
   
    #split positive and negative data into training and testing sets
    pos_train_ids, pos_test_ids = split_data(pos_review_ids)
    neg_train_ids, neg_test_ids = split_data(neg_review_ids)
    #add labels to the data and concatenate
    training = [(movie_reviews.words(f),'pos') for f in pos_train_ids]+[(movie_reviews.words(f),'neg') for f in neg_train_ids]
    testing = [(movie_reviews.words(f),'pos') for f in pos_test_ids]+[(movie_reviews.words(f),'neg') for f in neg_test_ids]
    #now normalise and create bag-of-words FreqDist representations
    training_norm=[(FreqDist(normalise(wordlist)),label) for (wordlist,label) in training]
    testing_norm=[(FreqDist(normalise(wordlist)),label) for (wordlist,label) in testing]
    return training_norm, testing_norm



In [18]:
random.seed(67)  #set the random seeds so these random splits are always the same
training,testing=get_train_test_data()


In [19]:
nb_classifier=NBClassifier()
nb_classifier.train(training)

In [20]:
docs,labels=zip(*testing)
cm=ConfusionMatrix(nb_classifier.classify_many(docs),labels,classes=('pos','neg'))
cm.precision()

0.8361774744027304

### NLTK NB Classifier
Developing our own NB classifier is great for understanding how it works.  But, in practice, it is usually more convenient to use a standard one imported from a library.  NLTK provides a NB classifier (as do other libraries such as sklearn).  It can be imported and trained as follows.


In [21]:
from nltk.classify import NaiveBayesClassifier

#note that the NaiveBayesClassifier.train() method is a class method which returns the classifier object.
#this is different to ours and other classifiers which are first instantiated and then trained via an object method
nltk_nb=NaiveBayesClassifier.train(training)

This object also has a .classify_many() method:

In [22]:
nltk_nb.classify_many(docs)

['pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'neg',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',


### Exercise 3 

Compare the performance of the NLTK NB classifier with the one that you wrote yourself.

In [23]:
cm=ConfusionMatrix(nltk_nb.classify_many(docs),labels,classes=('pos','neg'))
cm.precision()

0.6518847006651884

### Extension exercises
* Investigate the impact of the amount of training data on the Naive Bayes classifiers.
* Research what kind of event model is used by the NLTK NB classifier by default and whether / how this can be changed.  Does this impact the performance?
* Find out about a NB classifier provided by a different library e.g., sklearn.  Can you apply this to the movie_review data set?
* Find out about another machine learning method for classification (e.g., logistic regression).  Can you apply this to the movie_review data set?